# Dimension Reduction for Mixed Data (numerical and categorical) and K-Medoids Clustering with Gower Distance 

Traditional dimension reduction and clustering techniques cannot be applied directly to categorical data, because the notion of distance measure is unclear for these data. Most reduction and clustering techniques rely on measuring the closeness of observations (or usage of some distance metric in the algorithm), which cannot directly translate to categorical data. You cannot measure the distance between gender, nor color. At least not with any traditional approach.

Luckily, researchers have found (multiple) solutions to this problem. I will be using the well known housing data set on Kaggle. This is a simple data set, but it serves the purpose of this project. The details of this data set can be found [here](https://www.kaggle.com/c/house-prices-advanced-regression-techniques/overview).

I have removed some of the columns that contains too many missing values. I have also removed the remaining missing values. The goal again is to predict the housing price using the predictors.

Importing libraries

In [1]:
import numpy as np
import pandas as pd
from sklearn.decomposition import PCA
from prince import MCA
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split
from sklearn.linear_model import Ridge
import gower
from pyclustering.cluster.kmedoids import kmedoids
from sklearn.metrics.cluster import normalized_mutual_info_score

Loading the [housing dataset](https://www.kaggle.com/c/house-prices-advanced-regression-techniques/overview), sourced from Kaggle.

Doing a little bit of data preparation for the oncoming analysis:
* All features containing more that 30 missing observations are dropped.
* All missing values left are dropped, also.
* _Sale Price_ is the dependent variable $y$.

In [2]:
# Load data set
data = pd.read_csv('train.csv')
data = data.drop('Id', axis = 1)

# Remove columns that have too many missing values
data = data.drop(data.columns[data.isnull().sum() > 30], axis = 1)

# Remove missing values
data.dropna(inplace = True)

X = data.drop('SalePrice', axis=1)
y = data.SalePrice

## Part I: Dimension Reduction

For this first part, we will compare two regression models: a simple linear regression with PCA and MCA dimension reduction, and a Ridge regression. PCA will be used for the numerical features and MCA for the categorical ones.

Here's a roadmap of the plan:
1. Split the data into 3 pieces (training, validation, testing).
2. Make sure the categorical features on the training and validation sets are the same.
1. Tune the model.
2. Reapply the code, but on the training + validation sets combined, and the test set.
3. Fit the final model, perform prediction.

First, let's split the dataset into train, validation and test sets.

In [3]:
# HOLD OUT SET
# ------------
X_train_val, X_test, y_train_val, y_test = train_test_split(X, y, test_size=0.3, random_state=1)
X_train, X_valid, y_train, y_valid = train_test_split(X_train_val, y_train_val, test_size=0.25, random_state=1)

### Linear Regression with PCA and MCA

To split the dataset by data type, we first identify what types we have on the dataset.

In [4]:
X.dtypes.value_counts()

int64      33
object     28
float64     1
Name: count, dtype: int64

Then, we proceed with separating these features for both the training and validation sets.

In [5]:
# Separating categorical and numerical features
X_train_cat = X_train.select_dtypes(include=['object'])
X_train_num = X_train.select_dtypes(include=['int', 'float'])

# For valid test
X_valid_cat = X_valid.select_dtypes(include=['object'])
X_valid_num = X_valid.select_dtypes(include=['int', 'float'])


Before applying any MCA fitting and transform, we make sure that each categorical variables in the training set and validation set have the same set of levels. We'll remove any columns that don’t match.

In [6]:
# Validating that the training feature and testing feature have same number of levels
keep = X_train_cat.nunique() == X_valid_cat.nunique()
# Removing columns whose levels don't match
X_train_cat = X_train_cat[X_train_cat.columns[keep]]
X_valid_cat = X_valid_cat[X_valid_cat.columns[keep]]

# Making sure the classes are the same for categorical features that have same levels
keep = []
for i in range(X_train_cat.shape[1]):
    keep.append(all(np.sort(X_train_cat.iloc[:,i].unique()) == np.sort(X_valid_cat.iloc[:,i].unique())))
X_train_cat = X_train_cat[X_train_cat.columns[keep]]
X_valid_cat = X_valid_cat[X_valid_cat.columns[keep]]

X_valid = pd.concat([X_valid_cat, X_valid_num], axis=1)

##### _Training_

Searching for the best model using the training and validation sets.

In [7]:
# Define the parameter grid
n_components = [2, 7, 12, 17]

# Define variables to keep track of best parameter and score
best_score = float('inf')
best_params = {}

for pca_comp in n_components:
    for mca_comp in n_components:
        # Standardize numerical features
        scaler = StandardScaler()
        X_train_num_scaled = scaler.fit_transform(X_train_num)
        X_valid_num_scaled = scaler.fit_transform(X_valid_num)

        # Applying PCA on scaled numerical features
        pca = PCA(n_components=pca_comp)
        X_train_num_pca = pca.fit_transform(X_train_num_scaled)
        X_valid_num_pca = pca.transform(X_valid_num_scaled)
        # Keeping the original indexes
        X_train_num_pca = pd.DataFrame(data=X_train_num_pca, index=X_train_num.index)
        X_valid_num_pca = pd.DataFrame(data=X_valid_num_pca, index=X_valid_num.index)

        # Applying MCA on categorical features
        mca = MCA(n_components=mca_comp)
        X_train_cat_mca = mca.fit_transform(X_train_cat)
        X_valid_cat_mca = mca.transform(X_valid_cat)

        # Combining the result of PCA and MCA together
        X_train_reduced = pd.concat([pd.DataFrame(X_train_num_pca), pd.DataFrame(X_train_cat_mca)], axis=1)
        X_valid_reduced = pd.concat([pd.DataFrame(X_valid_num_pca), pd.DataFrame(X_valid_cat_mca)], axis=1)

        # Initialize and fit the linear regression model
        model = LinearRegression()
        model.fit(X_train_reduced, y_train)

        y_pred = model.predict(X_valid_reduced)

        mse = mean_squared_error(y_valid, y_pred)

        # Update the best score and parameters if this model is better
        if mse < best_score:
            best_score = mse
            best_params = {'pca_comp': pca_comp,
                           'mca_comp': mca_comp}

best_params


{'pca_comp': 2, 'mca_comp': 17}

##### _Performance_

After finding the optimal combinations of number of components for both PCA and MCA, we now evaluate the performance of the model using the testing set.

First, we have to prepare the data again. Now the training side will be composed of training and validation.

In [8]:
# Separating categorical and numerical features
X_train_val_cat = X_train_val.select_dtypes(include=['object'])
X_train_val_num = X_train_val.select_dtypes(include=['int', 'float'])

# For valid test
X_test_cat = X_test.select_dtypes(include=['object'])
X_test_num = X_test.select_dtypes(include=['int', 'float'])

Again, we make sure that each categorical variables in the training set and validation set have the same set of levels.

In [9]:
# Validating that the training feature and testing feature have same number of levels
keep = X_train_val_cat.nunique() == X_test_cat.nunique()
# Removing columns whose levels don't match
X_train_val_cat = X_train_val_cat[X_train_val_cat.columns[keep]]
X_test_cat = X_test_cat[X_test_cat.columns[keep]]

# Making sure the classes are the same for categorical features that have same levels
keep = []
for i in range(X_train_val_cat.shape[1]):
    keep.append(all(np.sort(X_train_val_cat.iloc[:,i].unique()) == np.sort(X_test_cat.iloc[:,i].unique())))
X_train_val_cat = X_train_val_cat[X_train_val_cat.columns[keep]]
X_test_cat = X_test_cat[X_test_cat.columns[keep]]

X_test = pd.concat([X_test_cat, X_valid_num], axis=1)

Using training+validation to fit the model, the using the testing set to predict and evaluate the model.

In [10]:
# Standardize numerical features
scaler = StandardScaler()
X_train_val_num_scaled = scaler.fit_transform(X_train_val_num)
X_test_num_scaled = scaler.fit_transform(X_test_num)

# Applying PCA on scaled numerical features
pca = PCA(n_components=best_params['pca_comp'])
X_train_val_num_pca = pca.fit_transform(X_train_val_num_scaled)
X_test_num_pca = pca.transform(X_test_num_scaled)
# Keeping the original indexes
X_train_val_num_pca = pd.DataFrame(data=X_train_val_num_pca, index=X_train_val_num.index)
X_test_num_pca = pd.DataFrame(data=X_test_num_pca, index=X_test_num.index)

# Applying MCA on categorical features
mca = MCA(n_components=best_params['mca_comp'])
X_train_val_cat_mca = mca.fit_transform(X_train_val_cat)
X_test_cat_mca = mca.transform(X_test_cat)

# Combining the result of PCA and MCA together
X_train_val_reduced = pd.concat([pd.DataFrame(X_train_val_num_pca), pd.DataFrame(X_train_val_cat_mca)], axis=1)
X_test_reduced = pd.concat([pd.DataFrame(X_test_num_pca), pd.DataFrame(X_test_cat_mca)], axis=1)

# Initialize and fit the linear regression model
model = LinearRegression()
model.fit(X_train_val_reduced, y_train_val)

y_pred = model.predict(X_test_reduced)

mse = mean_squared_error(y_test, y_pred)
print(f"Dimension Reduction Mean Squared Error: {mse}")

Dimension Reduction Mean Squared Error: 1030130708.3429468


### Ridge Regression

As a baseline model, we will use a Ridge regression model.

First, we start with splitting the dataset into training and testing. We concatenate the dataframes containing the numerical and caterogical features together.

In [11]:
X_train = pd.concat([X_train_val_num, pd.get_dummies(X_train_val_cat, drop_first=True)], axis=1)
X_test = pd.concat([X_test_num, pd.get_dummies(X_test_cat, drop_first=True)], axis=1)

We define the range of values used to tune the hyperparameter `alpha` ($lambda$) for the Ridge regression model.

With the model obtained by `GridSearchCV`, we make the predictions on the validation set.

Finally, we calculate and print the mean squared error for this model.

In [12]:
# Create a Ridge regression model
ridge = Ridge()

# Define a range of alpha values to tune
param_grid = {'alpha': [0.01, 0.1, 1.0, 10.0]}

# Create a GridSearchCV object with Ridge regression and alpha parameter grid
grid_search = GridSearchCV(estimator=ridge, param_grid=param_grid, scoring='neg_mean_squared_error', cv=5)

# Fit the grid search to the training data
grid_search.fit(X_train, y_train_val)

# Make predictions on the validation set
y_test_pred = grid_search.predict(X_test)

# Evaluate the model on the test set
ridge_mse = mean_squared_error(y_test, y_test_pred)
print(f"Ridge Regression Mean Squared Error: {ridge_mse}")


Ridge Regression Mean Squared Error: 937115200.3955582


| Model               | MSE                |
| ------------------- | ------------------ |
| Dimension Reduction | 1030130708.3429468 |
| Ridge               | 937115200.3955582  |

In summary, based on the MSE values you provided, the tuned Ridge regression model is outperforming the simple linear regression model with PCA and MCA dimension reduction in terms of predictive accuracy, as indicated by the lower MSE.


## Part II: Clustering Analysis

While we perform dimension reduction separately for numerical and categorical data, there are methods that can perform clustering analysis with numerical and categorical data combined. As usual, the most important aspect is the distance metric to use. For mixed data types, researchers have proposed to use the Gower distance. The Gower distance is essentially a special distance metric that measures numerical data and categorical data separately, then combine them to form a distance calculation. 

Calculating Gower Distance Matrix on the full predictors set.

In [13]:
g_matrix = gower.gower_matrix(X)

K-Medoids model parameters, in order:
1. `g_matrix`: calculated gower distance to feed the K-Medoids model.
2. `initial_medoids`: randomized initial medoids values, equivalent to the number of components.
3. `'distance_matrix'`: instructing the model that the input value is not raw data but a precalculated distance matrix.

Finally, we print the indexes of observations for each cluster. Below we print the indexes of the medoids.

In [14]:
n_components = 4
initial_medoids = np.random.randint(20, size=(1, n_components)).tolist()[0]

# Creating and fitting the K-Medoid model
kmedoids_instance = kmedoids(g_matrix, initial_medoids, data_type='distance_matrix')
kmedoids_instance.process()

# Clusters and Medoids
clusters = kmedoids_instance.get_clusters()
medoids = kmedoids_instance.get_medoids()

print(clusters)
print(medoids)

[[0, 6, 13, 18, 22, 25, 27, 32, 34, 36, 45, 47, 53, 60, 64, 67, 81, 82, 89, 96, 100, 103, 112, 117, 118, 123, 133, 141, 143, 148, 151, 162, 178, 186, 189, 192, 196, 199, 200, 203, 211, 213, 216, 219, 220, 224, 229, 235, 237, 239, 250, 256, 275, 277, 280, 281, 282, 283, 290, 301, 304, 308, 325, 331, 332, 335, 336, 337, 342, 354, 361, 377, 380, 384, 387, 388, 398, 399, 400, 411, 419, 420, 425, 427, 439, 442, 443, 456, 460, 464, 467, 471, 472, 473, 475, 477, 479, 480, 482, 495, 506, 514, 524, 528, 537, 538, 540, 550, 551, 557, 565, 579, 583, 591, 593, 595, 596, 602, 604, 608, 610, 611, 616, 629, 637, 638, 640, 642, 651, 661, 664, 667, 670, 674, 679, 680, 685, 687, 696, 697, 701, 703, 704, 709, 717, 718, 721, 723, 724, 727, 739, 741, 745, 749, 762, 765, 771, 772, 773, 779, 780, 787, 790, 802, 808, 812, 814, 816, 821, 822, 824, 835, 847, 848, 861, 863, 876, 877, 882, 885, 895, 903, 919, 920, 922, 925, 927, 929, 930, 931, 943, 954, 959, 976, 981, 988, 993, 996, 998, 1002, 1010, 1013, 1014, 1

Since the clustering result only indicate which observations belong to what cluster $k_i$, we store an observation's cluster number in the list `km_cluster`.

Below we bin the response variable into the numbers of categories used for k-Medoids (4).

In [15]:
# K-Medoids assigned cluster to each observation
km_cluster = []
for n_c, cluster in enumerate(clusters):
    for index in cluster:
        # lst.append([index, n_c])
        km_cluster.append(n_c)
    
# Ground truth
bins = pd.qcut(y, q=n_components, labels=False).tolist()

Now we calculate the Normalized Mutual Information (NMI)  between the K-Medoids clusters and the binning method.

In [16]:
normalized_mutual_info_score(km_cluster, bins)

0.0032549711961263605

The obtained NMI means that the clustering obatined by the response variable, SalePrice, is very dissimilar to the one performd by K-Medoids.

Since the ground truth used by this analysis is not ruled by true labels on the dataset, we cannot say if K-Medoids or the binning method is a better clustering criteria. We can only conclude that both models capture the relationships between features in a different manner.